Import basic libraries

In [1]:
from __future__ import division, print_function, absolute_import
from __future__ import unicode_literals

import numpy as np
import math

Import trajectories

In [2]:
import mdshare

### Note: Please replace the working_directory with the current one ! ###
wd = '/idiom/demo/' ### This defines the working directory

local_filename = mdshare.fetch('alanine-dipeptide-3x250ns-backbone-dihedrals.npz',working_directory=wd )
with np.load(local_filename) as fh:
    trajs = [fh[key] for key in fh.keys()]
    
local_filename1 = mdshare.fetch('alanine-dipeptide-3x250ns-heavy-atom-positions.npz',working_directory=wd )
with np.load(local_filename1) as fh:
    trajs1 = [fh[key] for key in fh.keys()]
    
traj_concat = np.concatenate((trajs[0],trajs[1],trajs[2]),axis=0)[::1,:]
print(traj_concat.shape)
data_size,_ = traj_concat.shape
# print(traj_concat[:10,:])

data_phi = traj_concat[:,0]
data_psi = traj_concat[:,1]
torsion = np.column_stack((data_phi,data_psi))

########## xyz:
traj_concat1 = np.concatenate((trajs1[0],trajs1[1],trajs1[2]),axis=0)[::1,:]

atom_position_list = np.split(traj_concat1,10,axis=1)
# print(atom_position_list[1][0])

dist_list = []
for i in range(10):
    for j in range(i+1,10):
        dist_tmp = np.sum(
            np.square(atom_position_list[i]-atom_position_list[j]),
        axis=-1)
        dist_list.append(dist_tmp)
        
print(len(dist_list))

dist_array_raw = np.column_stack(dist_list)
dist_array = (dist_array_raw - np.mean(dist_array_raw,axis=0,keepdims=True))/( np.std(dist_array_raw,axis=0,keepdims=True) ) ## Batch Normalization


(750000, 2)
45


Task Sampling by FPS:

In [3]:

def distance(A, B):
            return np.sqrt( np.sum( ( A - B )**2 , axis=-1) ) ### [N,s] or [N]
        
def MaxMinSampling(points, seed_id, n=1000):
    
    seed_id = seed_id.astype(np.int32)
    seed = points[ seed_id ]
    s,_ = seed.shape
    
    landmark = seed_id.tolist() ### [s], to be expanded
    
    pts_3d = np.expand_dims(points,axis=1) ## [N,1,dim]
    seed_3d = np.expand_dims(seed,axis=0) ## [1,s,dim]
    
    m = np.min( distance(pts_3d,seed_3d), axis=-1) ### [N]
    
    for i in range(s,n):
        lm = np.argmax(m)
        landmark.append( lm ) 
        ###
        dist = distance( np.expand_dims(points[lm],axis=0) , points ) ### [N]
        m = np.minimum( m, dist )
        
    return landmark


Perform mini-batch Farthest Point Sampling (FPS)

In [ ]:
win_id_list = []

n_iter = 10
minibs = data_size // n_iter
id_set = np.arange(minibs)

for i in range(n_iter):
    print('Iteration: ', i)
    np.random.shuffle(id_set)
    seed_id = id_set[:100]
    
    dist_array_tmp = dist_array[ i*minibs : (i+1)*minibs ]
    win_id = MaxMinSampling( dist_array_tmp, seed_id=seed_id, n=1000 ) ###minibs
    win_id = np.array(win_id).astype(np.int32)
    
    win_id_shifted = i*minibs + win_id
    win_id_list.append(win_id_shifted)
    
id_array = np.concatenate(win_id_list)
np.savetxt('./fps.txt', id_array .reshape((-1,1)),fmt='%d')
np.savetxt('./torsion.fps.txt',torsion[id_array])